<a href="https://colab.research.google.com/github/Sumegha22/Coronavirus-Tweet-Sentiment-Analysis-Classification-ML-Project/blob/main/SUMANA%7C%7C_Coronavirus_Tweets_Sentiment_Analysis_Classification_ML_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from wordcloud import WordCloud
import nltk
nltk.download('all',quiet=True)
from PIL import Image

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_file_path = "train.csv"
dev_file_path = "val.csv"
test_file_path = "test.csv"

tweet_train =pd.read_csv(train_file_path)
tweet_dev =pd.read_csv(dev_file_path)
tweet_test =pd.read_csv(test_file_path)

In [ ]:
tweet_train.head()

In [ ]:
tweet_train.shape

In [ ]:
tweet_train.columns

In [ ]:
tweet_train.index

In [ ]:
tweet_train.info()

In [ ]:
tweet_train.duplicated().sum()
tweet_dev.duplicated().sum()
tweet_test.duplicated().sum()

In [ ]:
tweet_train.isna().sum()
tweet_dev.isna().sum()
tweet_test.isna().sum()

In [ ]:
tweet_train.isnull().sum().sort_values(ascending=False)
tweet_dev.isnull().sum().sort_values(ascending=False)
tweet_test.isnull().sum().sort_values(ascending=False)

In [ ]:
tweet_train.columns
tweet_dev.columns
tweet_test.columns

In [ ]:
tweet_train.describe(include='all')
tweet_dev.describe(include='all')
tweet_test.describe(include='all')

In [ ]:
print(tweet_train.apply(lambda col: col.unique()))
print(tweet_dev.apply(lambda col: col.unique()))  
print(tweet_test.apply(lambda col: col.unique()))

In [ ]:
tweet_train.Sentiment.unique()
tweet_dev.Sentiment.unique()
tweet_test.Sentiment.unique()

In [ ]:
replace_values = {"Sentiment":{1:'Negative', 0:'Positive'}}
tweet_train = tweet_train.replace(replace_values)
tweet_dev = tweet_dev.replace(replace_values)
tweet_test = tweet_test.replace(replace_values)
tweet_train.head()

In [ ]:
sentiment_count = tweet_train['Sentiment'].value_counts().reset_index()
sentiment_count.columns = ['Sentiment','count']
sentiment_count

In [ ]:
plt.figure(figsize=(20,7))
ax = sns.barplot(x="Sentiment", y='count', data=sentiment_count)
ax.set_title("Proportion of Sentiment", fontsize=20)
ax.set_xlabel("Sentiment", fontsize=20)
ax.set_ylabel('count', fontsize=20)

In [ ]:
tweet_train.head()

In [ ]:
sentiment_count1 = tweet_train['Sentiment'].value_counts().reset_index()
sentiment_count1.columns = ['Sentiment','count']
sentiment_count1

In [ ]:
sentiment_count1 = tweet_train['Sentiment'].value_counts().to_list()
labels=['Positive','Negative']
plt.figure(figsize=(10,8))
plt.pie(x=sentiment_count1,explode=[0.04,0.04],shadow= True,labels=labels,autopct="%.2f%%",radius=1.1)
plt.title("Proportion Of Sentiment", fontsize=20)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
sentiment_count2 = tweet_dev['Sentiment'].value_counts().to_list()
labels=['Positive','Negative']
plt.figure(figsize=(10,8))
plt.pie(x=sentiment_count2,explode=[0.04,0.04],shadow= True,labels=labels,autopct="%.2f%%",radius=1.1)
plt.title("Proportion Of Sentiment", fontsize=20)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
sentiment_count3 = tweet_test['Sentiment'].value_counts().to_list()
labels=['Positive','Negative']
plt.figure(figsize=(10,8))
plt.pie(x=sentiment_count3,explode=[0.04,0.04],shadow= True,labels=labels,autopct="%.2f%%",radius=1.1)
plt.title("Proportion Of Sentiment", fontsize=20)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
tweet_train.isnull().sum().sort_values(ascending=False)
tweet_dev.isnull().sum().sort_values(ascending=False)
tweet_test.isnull().sum().sort_values(ascending=False)

In [ ]:
print(tweet_train[['OriginalTweet', 'Sentiment']].info())
print(tweet_dev[['OriginalTweet', 'Sentiment']].info())
print(tweet_test[['OriginalTweet', 'Sentiment']].info())

In [ ]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
import regex as re

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

def transform_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special entities, tickers, and URLs
    text = re.sub(r'\&\w*;|#\w*|@\w*', '', text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove small words, whitespace, and non-alphanumeric characters
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'\s\s+', ' ', text).strip()
    
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove non-alphanumeric characters
    words = [word for word in words if word.isalnum()]
    
    # Remove stopwords and punctuation
    stopwords_set = set(stopwords.words('english'))
    punctuation_set = set(string.punctuation)
    words = [word for word in words if word not in stopwords_set and word not in punctuation_set]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words into a string and return
    return ' '.join(lemmatized_words)


In [ ]:
tweet_train["Clean_Tweets"] = tweet_train['OriginalTweet'].apply(transform_text)
tweet_dev["Clean_Tweets"] = tweet_dev['OriginalTweet'].apply(transform_text)
tweet_test["Clean_Tweets"] = tweet_test['OriginalTweet'].apply(transform_text)

In [ ]:
tweet_train.iloc[56]["OriginalTweet"]
tweet_dev.iloc[56]["OriginalTweet"]
tweet_test.iloc[56]["OriginalTweet"]

In [ ]:
tweet_train.head()

In [ ]:
tweet_train['temp_list'] = tweet_train['Clean_Tweets'].apply(lambda x:str(x).split())
tweet_dev['temp_list'] = tweet_dev['Clean_Tweets'].apply(lambda x:str(x).split())
tweet_test['temp_list'] = tweet_test['Clean_Tweets'].apply(lambda x:str(x).split())

In [ ]:
tweet_train['temp_list']
tweet_dev['temp_list']
tweet_test['temp_list']

In [ ]:
X_train = tweet_train["Clean_Tweets"]  
y_train = tweet_train["Sentiment"]     

X_dev = tweet_dev["Clean_Tweets"]     
y_dev = tweet_dev["Sentiment"]         

X_test = tweet_test["Clean_Tweets"]    
y_test = tweet_test["Sentiment"]       

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bw_vectorizer = CountVectorizer(binary=False,max_df=1.0,min_df=5,ngram_range=(1,2))

bw_X_train = bw_vectorizer.fit_transform(X_train.astype(str).str.strip())
bw_X_dev = bw_vectorizer.transform(X_dev.astype(str).str.strip())
bw_X_test = bw_vectorizer.transform(X_test.astype(str).str.strip())

In [ ]:
lr_cv = LogisticRegression()
parameters = {
    'penalty': ['l1', 'l2'],  
    'C': [100, 10, 1.0, 0.1, 0.01]  
}

logreg_Gcv = GridSearchCV(lr_cv, parameters, cv=10)
logreg_Gcv.fit(bw_X_train, y_train)

y_dev_pred_lr_cv = logreg_Gcv.predict(bw_X_dev)
print("Validation Accuracy:", accuracy_score(y_dev, y_dev_pred_lr_cv))
print("Validation Classification Report:")
print(classification_report(y_dev, y_dev_pred_lr_cv))

y_test_pred_lr_cv = logreg_Gcv.predict(bw_X_test)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred_lr_cv))
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred_lr_cv))

In [ ]:
dt_cv = DecisionTreeClassifier()
cv_score_dt_cv = cross_val_score(dt_cv, bw_X_train, y_train, cv=5)
print("CV Score: {}" .format(np.mean(cv_score_dt_cv)))
dt_cv.fit(bw_X_train, y_train)

pred_dt_cv_dev = dt_cv.predict(bw_X_dev)
print("Validation Accuracy:", accuracy_score(y_dev, pred_dt_cv_dev))
print("Validation Classification Report:")
print(classification_report(y_dev, pred_dt_cv_dev))

pred_dt_cv_test = dt_cv.predict(bw_X_test)
print("Test Accuracy:", accuracy_score(y_test, pred_dt_cv_test))
print("Test Classification Report:")
print(classification_report(y_test, pred_dt_cv_test))

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

In [ ]:
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
mnb.fit(bw_X_train, y_train)
y_pred2_dev = mnb.predict(bw_X_dev)
accuracy_mnb_dev = accuracy_score(y_dev, y_pred2_dev)
print("Validation Accuracy: ")
print(accuracy_mnb_dev)
print("Validation Classification Report:")
print(classification_report(y_dev, y_pred2_dev))

y_pred2_test = mnb.predict(bw_X_test)
accuracy_mnb_test = accuracy_score(y_test, y_pred2_test)
print("Test Accuracy: ")
print(accuracy_mnb_test)
print("Test Classification Report:")
print(classification_report(y_test, y_pred2_test))

In [ ]:
bnb.fit(bw_X_train, y_train)

y_pred3_dev = bnb.predict(bw_X_dev)
accuracy_bnb_dev = accuracy_score(y_dev, y_pred3_dev)
print("Validation Accuracy: ")
print(accuracy_bnb_dev)
print("Validation Classification Report:")
print(classification_report(y_dev, y_pred3_dev))

y_pred3_test = bnb.predict(bw_X_test)
accuracy_bnb_test = accuracy_score(y_test, y_pred3_test)
print("Test Accuracy: ")
print(accuracy_bnb_test)
print("Test Classification Report:")
print(classification_report(y_test,y_pred3_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,9]}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5)
grid_search.fit(bw_X_train, y_train)
print('Best hyperparameters:', grid_search.best_params_)
print('Mean cross-validation score:', grid_search.best_score_)
pred_knn_cv = grid_search.predict(bw_X_test)
pred_knn_cv
accuracy_KNN = accuracy_score(y_test, pred_knn_cv)
print("Accuracy :",(accuracy_KNN))
print(classification_report(y_test, pred_knn_cv))

In [ ]:
svm_cv = SVC()
svm_cv.fit(bw_X_train,y_train)

pred_svm_cv = svm_cv.predict(bw_X_test)

pred_svm_cv

accuracy_svc = accuracy_score(y_test,pred_svm_cv)
print("Accuracy :",(accuracy_svc))

print(classification_report(y_test,pred_svm_cv))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

rf_clf =RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(bw_X_train, y_train)
y_pred_rf = rf_clf.predict(bw_X_test)
accuracy_rf = (y_pred_rf == y_test).mean()
print('Accuracy_rf:', accuracy_rf)
print(classification_report(y_test,y_pred_rf))

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, random_state=42, max_iter=1000)
sgd_clf.fit(bw_X_train, y_train)
y_pred_sgd = sgd_clf.predict(bw_X_test)
accuracy_sgd= accuracy_score(y_test, y_pred_sgd)
print('Accuracy_sgd:', accuracy_sgd)
print(classification_report(y_test,y_pred_sgd))

In [ ]:
X_train = tweet_train["Clean_Tweets"]
y_train = tweet_train["Sentiment"]

X_dev = tweet_dev["Clean_Tweets"]
y_dev = tweet_dev["Sentiment"]

X_test = tweet_test["Clean_Tweets"]
y_test = tweet_test["Sentiment"]

In [ ]:
print(X_train.shape)
y_test.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=1.0, min_df=5, ngram_range=(1, 2), sublinear_tf=True)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.astype(str).str.strip())
X_dev_tfidf = tfidf_vectorizer.transform(X_dev.astype(str).str.strip())
X_test_tfidf = tfidf_vectorizer.transform(X_test.astype(str).str.strip())

In [ ]:
lr_model = LogisticRegression()
parameters = dict(penalty=['l1', 'l2'],C=[100, 10, 1.0, 0.1, 0.01])
lr_tv_Gcv=GridSearchCV(lr_model,parameters,cv=10)
lr_tv_Gcv.fit(X_train_tfidf, y_train)
y_dev_pred = lr_tv_Gcv.predict(X_dev_tfidf)
print("Validation Accuracy:", accuracy_score(y_dev, y_dev_pred))
print("Validation Classification Report:")
print(classification_report(y_dev, y_dev_pred))

y_test_pred = lr_tv_Gcv.predict(X_test_tfidf)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

In [ ]:
dt_tv = DecisionTreeClassifier()
dt_tv.fit(X_train_tfidf, y_train)
pred_dt_tv = dt_tv.predict(X_test_tfidf)
pred_dt_tv

cv_score_dt_tv = cross_val_score(dt_tv, X_train_tfidf, y_train, cv=5)
print("Accuracy: {}" .format(np.mean(cv_score_dt_tv)))
print(classification_report(y_test, pred_dt_tv))


In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
mnb.fit(X_train_tfidf,y_train)
y_pred2_mnb_tv = mnb.predict(X_test_tfidf)
accuracy_mnb_tv = accuracy_score(y_test, y_pred2_mnb_tv)
print(accuracy_score(y_test, y_pred2_mnb_tv))
print(classification_report(y_test, y_pred2_mnb_tv))

In [ ]:
bnb.fit(X_train_tfidf,y_train)
y_pred3_bnb_tv = bnb.predict(X_test_tfidf)
accuracy_bnb_tv = accuracy_score(y_test, y_pred3_bnb_tv)
print(accuracy_score(y_test,y_pred3_bnb_tv))
print(classification_report(y_test,y_pred3_bnb_tv))

In [ ]:
knn = KNeighborsClassifier()
param = {'n_neighbors': [1,2,3,4,5,6,7,8]}
knn_tv = GridSearchCV(estimator=knn,param_grid=param)

knn_tv.fit(X_train_tfidf, y_train)

pred_knn_tv = knn_tv.predict(X_test_tfidf)
pred_knn_tv
accuracy_KNN_tv = accuracy_score(y_test,pred_knn_tv)
print("Accuracy :",(accuracy_KNN_tv))
print(classification_report(y_test,pred_knn_tv))

In [ ]:
svm_cv = SVC()

svm_cv.fit(X_train_tfidf,y_train)

pred_svm_cv = svm_cv.predict(X_test_tfidf)

accuracy_svc = accuracy_score(y_test,pred_svm_cv)
print("Accuracy : ",(accuracy_svc))
print(classification_report(y_test,pred_svm_cv))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
rf_clf =RandomForestClassifier(n_estimators=100, random_state=42)

rf_clf.fit(X_train_tfidf, y_train)

y_pred_rf_tv = rf_clf.predict(X_test_tfidf)

accuracy_tv_rf = (y_pred_rf_tv == y_test).mean()
print('Accuracy_tv_rf:', accuracy_tv_rf)
print(classification_report(y_test,y_pred_rf_tv))

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, random_state=42, max_iter=1000)

sgd_clf.fit(X_train_tfidf, y_train)

y_pred_sgd_tv = sgd_clf.predict(X_test_tfidf)

accuracy_sgd_tv= accuracy_score(y_test, y_pred_sgd_tv)
print('Accuracy:', accuracy_sgd_tv)
print(classification_report(y_test,y_pred_sgd_tv ))